In [1]:
import pandas as pd

In [2]:
# via https://www.worldometers.info/world-population/population-by-country/
tables = pd.read_html("SciHubData/Population-by-Country-2022-Worldometer.html")
fullcdata = tables[0]
fullcdata

,#,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,China,1439323776,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,2,India,1380004385,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,3,United States,331002651,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,4,Indonesia,273523615,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,5,Pakistan,220892340,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...,...
230,231,Montserrat,4992,0.06 %,3,50,100,NaN,N.A.,N.A.,10 %,0.00 %
231,232,Falkland Islands,3480,3.05 %,103,0,12170,NaN,N.A.,N.A.,66 %,0.00 %
232,233,Niue,1626,0.68 %,11,6,260,NaN,N.A.,N.A.,46 %,0.00 %
233,234,Tokelau,1357,1.27 %,17,136,10,NaN,N.A.,N.A.,0 %,0.00 %


In [3]:
cdata = fullcdata[["Country (or dependency)", "Population (2020)"]] \
  .rename(
    columns={
        "Country (or dependency)": "country",
        "Population (2020)": "population",
    }
)
cdata

,country,population
0,China,1439323776
1,India,1380004385
2,United States,331002651
3,Indonesia,273523615
4,Pakistan,220892340
...,...,...
230,Montserrat,4992
231,Falkland Islands,3480
232,Niue,1626
233,Tokelau,1357


In [4]:
# last month's downloads by country https://sci-hub.se/stats
scidata = pd.read_excel("SciHubData/sci_hub_downloads_by_country_feb2022.ods", sheet_name="Sheet1")
scidata

,rank,country,articles
0,1,China,337184513
1,2,United States,126753744
2,3,France,80404111
3,4,Brazil,40278282
4,5,India,25296581
5,6,Hong Kong,19295749
6,7,Indonesia,17171446
7,8,Germany,15117965
8,9,Mexico,11334419
9,10,Iran,10875894


## Join the two data tables

In [5]:
# # ALT. use the join method
# scidata.join(cdata.set_index("country"), how="left", on="country")

# # ERROR: because Pandas join uses cdata index (ints) by default
# scidata.join(cdata, how="left", on="country")

In [6]:
df = pd.merge(scidata, cdata, on="country", how="left")
df

,rank,country,articles,population
0,1,China,337184513,1439323776
1,2,United States,126753744,331002651
2,3,France,80404111,65273511
3,4,Brazil,40278282,212559417
4,5,India,25296581,1380004385
5,6,Hong Kong,19295749,7496981
6,7,Indonesia,17171446,273523615
7,8,Germany,15117965,83783942
8,9,Mexico,11334419,128932753
9,10,Iran,10875894,83992949


In [7]:
df2 = df \
  .assign(artperpop=df["articles"]/df["population"]) \
  .sort_values("artperpop", ascending=False)
df2["relrank"] = df2["artperpop"].rank(ascending=False)
df2[['relrank', 'country', 'artperpop', 'articles', 'rank', 'population']].reset_index(drop=True)

,relrank,country,artperpop,articles,rank,population
0,1.0,Hong Kong,2.573803,19295749,6,7496981
1,2.0,Singapore,1.259289,7367274,17,5850342
2,3.0,France,1.231803,80404111,3,65273511
3,4.0,Taiwan,0.384115,9148370,12,23816775
4,5.0,United States,0.382939,126753744,2,331002651
5,6.0,Netherlands,0.347248,5950058,19,17134872
6,7.0,Poland,0.240070,9085822,13,37846611
7,8.0,China,0.234266,337184513,1,1439323776
8,9.0,Canada,0.202599,7646508,16,37742154
9,10.0,Brazil,0.189492,40278282,4,212559417


Hong Kong, Singapore, and France are the top three.
These people like their research!